In [181]:
# Start by importing the necessary libraries
import numpy as np
import re
import nltk
import glob
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import stopwords
nltk.download('wordnet')
from textblob import Word
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import wordcloud
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import keras_metrics
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.models import Sequential
from keras.models import load_model
from keras import preprocessing
from keras.preprocessing import sequence
from collections import Counter
import wordcloud
import os
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from sklearn.feature_extraction.text import CountVectorizer
from keras.utils.np_utils import to_categorical
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import confusion_matrix, classification_report

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [182]:
# Load the dataset of app reviews
df = pd.read_csv('Monzo.csv')
df.head(5)

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,129510a0-b042-45f4-9f0a-3c497014652d,Jose Teles,https://play-lh.googleusercontent.com/a/AEdFTp...,"The only problem is when you cash out ,cash ma...",3,0,5.5.0,2022-12-23 21:09:04,Hey Jose 👋 Thanks for taking the time to leave...,2022-12-24 10:04:55
1,1616dc35-a879-43c6-bc7b-183388e4cde2,Manchu_uk,https://play-lh.googleusercontent.com/a/AEdFTp...,"It takes age's to receive card's, i have appli...",1,0,5.4.0,2022-12-23 20:43:21,Hi there 👋 Thanks for taking the time to post ...,2022-12-18 08:23:53
2,d5f8ad28-78ae-446f-85d1-f6c1a2d1f0ef,Patient Onyealusi,https://play-lh.googleusercontent.com/a/AEdFTp...,I've not gotten my Account Number,3,0,5.5.0,2022-12-23 17:10:06,Hey there 👋 Thanks for your review! Sorry for ...,2022-12-24 09:48:35
3,e219a36b-cad0-4eed-bce3-6687587d0efc,mutungi marvin,https://play-lh.googleusercontent.com/a-/AD5-W...,I was denied an account with no explanation wh...,1,0,5.5.0,2022-12-23 07:20:05,Hi there 👋 We're sorry to hear that 😔 We'll as...,2022-12-23 07:24:34
4,ad2dd2fd-67fc-4423-bf94-15a2f98cff86,M Absar Asif,https://play-lh.googleusercontent.com/a-/AD5-W...,Not a good experience with the app. And also t...,1,0,NaN,2022-12-23 06:52:20,Hi there 👋 Thanks for your feedback - we're so...,2022-12-23 06:57:57


In [183]:
#Checking for the shape of the dataset
df.shape

(15031, 10)

In [184]:
#Removing duplicates from the dataset
df.drop_duplicates("content", inplace=True)

In [185]:
#Checking for the shape of the dataset after removing duplicates
df.shape

(12864, 10)

In [186]:
#Checking for null values
df.isnull().sum()

reviewId                    0
userName                    0
userImage                   0
content                     1
score                       0
thumbsUpCount               0
reviewCreatedVersion     1393
at                          0
replyContent            10636
repliedAt               10636
dtype: int64

In [187]:
#Creating new dataset for only content and score
topic_df = df[['content', 'score']]
topic_df

,content,score
0,"The only problem is when you cash out ,cash ma...",3
1,"It takes age's to receive card's, i have appli...",1
2,I've not gotten my Account Number,3
3,I was denied an account with no explanation wh...,1
4,Not a good experience with the app. And also t...,1
...,...,...
15026,It's a waiting list. Wow! Absolutely pointless...,1
15027,Pretty disappointed to go from an email saying...,2
15028,On the waiting list. Looking forward to all th...,5
15029,Mondo for Android - can't wait. The iOS app i...,5


In [188]:
#Checking for the shape of the new dataset
topic_df.shape

(12864, 2)

In [189]:
#Checking for null values from the new dataset
check_nan = topic_df[topic_df.isna().any(axis=1)]
check_nan

,content,score
4662,NaN,5


In [190]:
#Dropping null values from the new dataset
topic_df = topic_df.dropna()

In [191]:
#Checking the shape of the dataset after dropping the null values
topic_df.shape

(12863, 2)

In [192]:
#Checkign the infor of the dataset
topic_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12863 entries, 0 to 15030
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  12863 non-null  object
 1   score    12863 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 301.5+ KB


In [193]:
#Creating a new column called sentiment by using the review rating score
def sentiment(n):return 1 if n >= 3 else 0
topic_df['sentiment'] = topic_df['score'].apply(sentiment)

C:\Users\USER\AppData\Local\Temp\ipykernel_18096\1701722460.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topic_df['sentiment'] = topic_df['score'].apply(sentiment)


In [194]:
#Printing from the new dataframe
topic_df.head(5)

,content,score,sentiment
0,"The only problem is when you cash out ,cash ma...",3,1
1,"It takes age's to receive card's, i have appli...",1,0
2,I've not gotten my Account Number,3,1
3,I was denied an account with no explanation wh...,1,0
4,Not a good experience with the app. And also t...,1,0


In [195]:
#Tokenizing, removing stop words, lemmatizing
def preprocess_df(df):
    # Create a new column to store the processed text
    df['processed_text'] = ''

    # Create a tokenizer and stemmer
    tokenizer = PunktSentenceTokenizer()
    stemmer = PorterStemmer()
    lemmatizer = nltk.WordNetLemmatizer()

    # Get a list of English stop words
    stop_words = set(stopwords.words('english'))

    # Iterate over the rows of the DataFrame
    for index, row in df.iterrows():
        # Tokenize the text
        tokens = tokenizer.tokenize(row['content'])

        # Lowercase the text
        tokens = [token.lower() for token in tokens]

        # Remove punctuation
        tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]

        # Remove stop words
        tokens = [token for token in tokens if token not in stop_words]

        # Stem the tokens
        tokens = [stemmer.stem(token) for token in tokens]

        # Lemmatize the tokens
        tokens = [lemmatizer.lemmatize(token) for token in tokens]

        # Join the tokens back into a single string
        processed_text = ' '.join(tokens)

        # Update the 'processed_text' column for the current row
        df.at[index, 'processed_text'] = processed_text

    return df

df_clean = preprocess_df(topic_df)
df_clean


C:\Users\USER\AppData\Local\Temp\ipykernel_18096\1540386533.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['processed_text'] = ''


,content,score,sentiment,processed_text
0,"The only problem is when you cash out ,cash ma...",3,1,the only problem is when you cash out cash mac...
1,"It takes age's to receive card's, i have appli...",1,0,it takes ages to receive cards i have applied ...
2,I've not gotten my Account Number,3,1,ive not gotten my account numb
3,I was denied an account with no explanation wh...,1,0,i was denied an account with no explanation wh...
4,Not a good experience with the app. And also t...,1,0,not a good experience with the app and also th...
...,...,...,...,...
15026,It's a waiting list. Wow! Absolutely pointless...,1,0,its a waiting list wow absolutely pointless at...
15027,Pretty disappointed to go from an email saying...,2,0,pretty disappointed to go from an email saying...
15028,On the waiting list. Looking forward to all th...,5,1,on the waiting list looking forward to all the...
15029,Mondo for Android - can't wait. The iOS app i...,5,1,mondo for android cant wait the ios app is fa...


In [196]:
# Extracting the text of the reviews and the corresponding ratings
texts = df_clean['processed_text'].values
labels = df_clean['sentiment'].values

In [197]:
#Dipslaying the content
texts

array(['the only problem is when you cash out cash machine charge you a fe',
       'it takes ages to receive cards i have applied 10 dats ago and they said i will receive on 23 days im not going to use monzo even if the send me card edit its been 16 days and not yet received i applied for another bank like last week ago they sent me everything and even internet banking is access',
       'ive not gotten my account numb', ...,
       'on the waiting list looking forward to all the new features to com',
       'mondo for android  cant wait the ios app is fantast if android is only half as good would be great',
       'it is just the waiting list for now but it is a first step i cant wait to get my mondo account'],
      dtype=object)

In [198]:
#Dipslaying the labels
labels

array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [199]:
# Tokenize the texts
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [200]:
# Get the word index
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 10753 unique tokens.


In [201]:
# Pad the sequences to the same length
max_length = 150
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [202]:
# Split the data into train and test sets
test_size = 0.2
random_state = 42
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=test_size, random_state=random_state)

In [203]:
# Build the model
model = Sequential()
model.add(Embedding(max_words, 50, input_length=max_length))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [204]:
# model summary
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 150, 50)           500000    
                                                                 
 lstm_6 (LSTM)               (None, 64)                29440     
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 529,505
Trainable params: 529,505
Non-trainable params: 0
_________________________________________________________________


In [205]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32)

Epoch 1/5
322/322 [==============================] - 109s 309ms/step - loss: 0.3431 - accuracy: 0.8474 - val_loss: 0.2294 - val_accuracy: 0.9098
Epoch 2/5
322/322 [==============================] - 138s 429ms/step - loss: 0.1773 - accuracy: 0.9330 - val_loss: 0.2373 - val_accuracy: 0.9075
Epoch 3/5
322/322 [==============================] - 155s 482ms/step - loss: 0.1289 - accuracy: 0.9540 - val_loss: 0.2459 - val_accuracy: 0.9087
Epoch 4/5
322/322 [==============================] - 128s 398ms/step - loss: 0.1008 - accuracy: 0.9659 - val_loss: 0.2757 - val_accuracy: 0.9106
Epoch 5/5
322/322 [==============================] - 116s 359ms/step - loss: 0.0855 - accuracy: 0.9711 - val_loss: 0.3106 - val_accuracy: 0.9036


In [206]:
predictions = model.predict(X_test)

81/81 [==============================] - 9s 75ms/step


In [207]:
# Convert the predictions to a binary classification (positive or negative)
predictions = (predictions > 0.5).astype(int)

In [208]:
# Print the confusion matrix
print(confusion_matrix(y_test, predictions))

[[ 548  117]
 [ 131 1777]]


In [209]:
#Getting the Accuracy score
print('Accuracy: %.3f' % accuracy_score(y_test, predictions))

Accuracy: 0.904


In [210]:
#Getting the F1 score
print('F1 score: %f'  % f1_score(y_test,predictions))

F1 score: 0.934771


In [211]:
# Print the classification report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.81      0.82      0.82       665
           1       0.94      0.93      0.93      1908

    accuracy                           0.90      2573
   macro avg       0.87      0.88      0.88      2573
weighted avg       0.90      0.90      0.90      2573

